In [72]:
from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
import numpy as np
import matplotlib.pyplot as plt

In [73]:
VERSION = 'v1.0-mini'
DATA_DIR = '/media/jim/Hard Disk/nuscenes_data/sets/nuscenes'
sensor = 'CAM_FRONT'
render = True
nusc = NuScenes(version=VERSION, dataroot=DATA_DIR, verbose=True)
nusc_can = NuScenesCanBus(dataroot=DATA_DIR)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.330 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [74]:
scene = nusc.scene[0]
scene_name = scene['name']
print(f"processing {scene_name}, ")
print(f"{scene['description']}")

processing scene-0061, 
Parked truck, construction, intersection, turn left, following a van


In [93]:
sample_tokens_list = []
timestamps = []
files_list = []
current_sample = nusc.get('sample', scene['first_sample_token'])
cam_front_data = nusc.get('sample_data', current_sample['data'][sensor])
calib_sensor = nusc.get('calibrated_sensor', cam_front_data['calibrated_sensor_token'])
camera_intrinsic = calib_sensor['camera_intrinsic']
while not current_sample['next'] == "":
    # print(f"current sample token: {current_sample['token']}")
    sample_tokens_list.append(current_sample['token'])
    cam_front_data = nusc.get('sample_data', current_sample['data'][sensor])
    assert(cam_front_data['calibrated_sensor_token'] == calib_sensor['token'])
    ego_pose = nusc.get('ego_pose', cam_front_data['ego_pose_token'])
    filename = cam_front_data['filename']
    timestamp = int(ego_pose['timestamp'])
    timestamps.append(timestamp)
    files_list.append(filename)
    # ego_R = ego_pose['rotation']
    # ego_T = ego_pose['translation']
    current_sample = nusc.get('sample', current_sample['next'])
timestamps = np.array(timestamps)
files_list = np.array(files_list)
camera_files = np.stack((timestamps, files_list)).T

# get imu and velocities
ms_imu = nusc_can.get_messages(scene_name, 'ms_imu') # 100hz
ms_imu_a_r = np.array([(m['utime'], m['linear_accel'][0], m['linear_accel'][1], m['linear_accel'][2],
                        m['rotation_rate'][0], m['rotation_rate'][1], m['rotation_rate'][2])
                       for m in ms_imu])
# steer angle!
angle = nusc_can.get_messages(scene_name, 'steeranglefeedback')
angle = np.array([(m['utime'], m['value']) for m in angle])

# Zoe Vehicle Info, 100hz
zoe_veh = nusc_can.get_messages(scene_name, 'zoe_veh_info')
wheel_speed = np.array([(m['utime'], m['FL_wheel_speed']) for m in zoe_veh])
# Convert to m/s.
radius = 0.305  # Known Zoe wheel radius in meters.
circumference = 2 * np.pi * radius
wheel_speed[:, 1] *= circumference / 60


In [98]:
# look at the times for the tokens in the current_sample

def analyze_series(timed_array):
    last_time = int(timed_array[-1])
    first_time = int(timed_array[0])
    print("duration of scene:")
    print((last_time - first_time) / (1e6))
    print("number of entries:")
    print(len(timed_array))
    print("averaged time diff in miliseconds:")
    print((last_time - first_time) / len(timed_array))
    print("total frequency:")  #  -> close enough to 100 hz!!
    print(len(timed_array) / (last_time - first_time) * (1e6))

print(">>  camera files ===")
print(analyze_series(camera_files[:, 0]))

>>  camera files ===
duration of scene:
18.65
number of entries:
38
averaged time diff in miliseconds:
490789.4736842105
total frequency:
2.037533512064343
None


In [99]:
# looking at the imu measurement times
# time duration, originally in microseconds -> seconds
print(">>  IMU scene stats ===")
print(analyze_series(ms_imu_a_r[:, 0]))
# looking at the steer angles
print(">>  steer angle stats ===")
print(analyze_series(angle[:, 0]))
# looking at the speed from wheel
print(">>  wheel speed stats ===")
print(analyze_series(wheel_speed[:, 0]))

>>  IMU scene stats ===
duration of scene:
19.149252
number of entries:
1899
averaged time diff in miliseconds:
10083.860979462876
total frequency:
99.1683643831101
None
>>  steer angle stats ===
duration of scene:
19.144668
number of entries:
1797
averaged time diff in miliseconds:
10653.682804674458
total frequency:
93.8642550500223
None
>>  wheel speed stats ===
duration of scene:
19.148904
number of entries:
1904
averaged time diff in miliseconds:
10057.197478991597
total frequency:
99.4312781556584
None


In [100]:
def closest_timestep_interpolate(reference_array, interpolate_array):
    match_array = []
    interp_idx = 0
    for ref_idx, entry in enumerate(reference_array):
        i_timestamp, i_value = entry
        i_timestamp = int(i_timestamp)
        if i_timestamp < interpolate_array[interp_idx][0]:
            print(f"skipping timestep {i_timestamp}")
            reference_array = reference_array[ref_idx+1:]
            continue
        time_diff = np.abs(interpolate_array[:, 0] - i_timestamp)
        closest_match = np.argmin(time_diff)
        match_array.append(interpolate_array[closest_match])
        # print(f"found match ref {ref_idx} interp {closest_match}")
    return reference_array, np.array(match_array)

# lining up doesn't work. the difference in times drifts. closest_neighbor interpolate
updated_files, updated_speed = closest_timestep_interpolate(reference_array=camera_files, interpolate_array=wheel_speed)
updated_files, updated_imu = closest_timestep_interpolate(reference_array=updated_files, interpolate_array=ms_imu_a_r)
updated_files, updated_angle = closest_timestep_interpolate(reference_array=updated_files, interpolate_array=angle)

skipping timestep 1532402927612460


In [101]:
len(updated_files), len(updated_angle), len(updated_speed), len(updated_imu)

(37, 37, 37, 37)

In [115]:
stitched_output = np.hstack((updated_files, updated_angle[:, 1:], updated_speed[:, 1:], updated_imu[:, 1:]))
stitched_output.shape

(37, 10)

In [116]:
stitched_output

array([['1532402928112460',
        'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402928112460.jpg',
        '0.05235987755982989', '8.791728034435481',
        '-0.6653889417648315', '-0.6056896448135376',
        '9.712173461914062', '0.018220271915197372',
        '0.017773376777768135', '0.015888486057519913'],
       ['1532402928662460',
        'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402928662460.jpg',
        '0.061086523819801536', '8.581291277968159',
        '-0.6841903924942017', '-0.40256914496421814',
        '10.334538459777832', '0.011411811225116253',
        '-0.05556841939687729', '0.009379098191857338'],
       ['1532402929162460',
        'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402929162460.jpg',
        '0.06283185307179427', '8.185723450937807',
        '-1.1970546245574951', '0.015408070757985115',
        '9.791049003601074', '-0.014990320429205894',
        '0.018028978258371353', '0.008693818

In [117]:
stitched_output[0][1]

'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402928112460.jpg'

In [119]:
th = updated_angle[0, 1]
v = updated_speed[0, 1]
v, th

(8.791728034435481, 0.05235987755982989)

In [124]:
xcomp = np.cos(th)
ycomp = -np.sin(th)
v*xcomp, v*ycomp

(8.779679276717049, -0.46012349371007794)

In [129]:
np.cos(updated_angle[:, 1])

array([ 0.99862953,  0.9981348 ,  0.99802673,  0.99802673,  0.99802673,
        0.99802673,  0.99744078,  0.99663739,  0.99357186,  0.97667228,
        0.91706007,  0.82708057,  0.70587157,  0.2890318 ,  0.03141076,
       -0.02443218, -0.15643447, -0.21303039, -0.38590604, -0.4539905 ,
       -0.45554491, -0.42103581, -0.27060045, -0.19765734,  0.12533323,
        0.31233492,  0.37784079,  0.7033947 ,  0.94205745,  0.99074784,
        0.99912283,  0.99939083,  0.99254615,  0.83676431,  0.10799936,
       -0.27563736, -0.6534206 ])

In [132]:
updated_angle[:, 1]

array([0.05235988, 0.06108652, 0.06283185, 0.06283185, 0.06283185,
       0.06283185, 0.0715585 , 0.08203047, 0.1134464 , 0.21642083,
       0.41015237, 0.5969026 , 0.78714349, 1.27758101, 1.5393804 ,
       1.59523094, 1.72787596, 1.78547182, 1.96698607, 2.04203522,
       2.04378055, 2.00538331, 1.84481302, 1.76976386, 1.44513262,
       1.2531464 , 1.18333323, 0.79063415, 0.34208453, 0.13613568,
       0.0418879 , 0.03490659, 0.12217305, 0.57944931, 1.46258591,
       1.85004901, 2.28289066])